# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# импорт библиотек
import pandas as pd
from pymystem3 import Mystem

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/data.csv')

In [3]:
# получение первых 10 строк таблицы df
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Согласно документации к данным:**
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

В названиях колонок нарушений не выявлено.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

Колонка `days_employed`, судя по описанию, должна быть целочисленной и положительной,
остальные колонки определились правильно.
Такой точности для `total_income` не требуется, можем преобразовать в целочисленный формат.

**Вывод**

В каждой строке таблицы — данные о заемщиках. Данные рассказывают о заемщике: возраст, образование, занятость, семейное положение и количество детей, доход, а также цель получения кредита.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [6]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [7]:
df[(df['days_employed'].isna() == True) & (df['total_income'].isna() == True)]['children'].count()

2174

#### Обработка пропусков `days_employed`

In [8]:
# подсчёт пропусков до обработки
print('Пропуски до:', df['days_employed'].isna().sum())

# фрейм с группированными значениями
medians = (df.groupby(['family_status_id', 'education_id'])
             .agg({'days_employed': 'median'})
             .rename(columns={'days_employed': 'median_days_employed'}))

# приджойним медиану к исходному датафрейму
df = df.merge(medians, on=['family_status_id', 'education_id'])

# в результате получим дополнительную колонку median_days_employed
df[['family_status', 'education', 'days_employed', 'median_days_employed']][df['days_employed'].isna()].head()

Пропуски до: 2174


,family_status,education,days_employed,median_days_employed
6,женат / замужем,высшее,NaN,-1443.773383
9,женат / замужем,высшее,NaN,-1443.773383
12,женат / замужем,высшее,NaN,-1443.773383
13,женат / замужем,высшее,NaN,-1443.773383
15,женат / замужем,высшее,NaN,-1443.773383


In [9]:
# те значения где days_employed пустой заполним значениями из median_days_employed
df.loc[df['days_employed'].isna(),'days_employed'] = df.loc[df['days_employed'].isna(),'median_days_employed']

# подсчёт пропусков после обработки
print('Пропуски после:', df['days_employed'].isna().sum())

# удалим расчетный столбец
df = df.drop('median_days_employed', 1)

Пропуски после: 0


#### Обработка пропусков `total_income`

In [10]:
# подсчёт пропусков до обработки
print('Пропуски до:', df['total_income'].isna().sum())

# фрейм с группированными значениями
medians = (df.groupby(['family_status_id', 'education_id'])
             .agg({'total_income': 'median'})
             .rename(columns={'total_income': 'median_total_income'}))

# приджойним медиану к исходному датафрейму
df = df.merge(medians, on=['family_status_id', 'education_id'])

# в результате получим дополнительную колонку median_total_income
df[['family_status', 'education', 'total_income', 'median_total_income']][df['total_income'].isna()].head()

Пропуски до: 2174


,family_status,education,total_income,median_total_income
6,женат / замужем,высшее,NaN,177914.29425
9,женат / замужем,высшее,NaN,177914.29425
12,женат / замужем,высшее,NaN,177914.29425
13,женат / замужем,высшее,NaN,177914.29425
15,женат / замужем,высшее,NaN,177914.29425


In [11]:
# те значения где days_employed пустой заполним значениями из median_days_employed
df.loc[df['total_income'].isna(),'total_income'] = df.loc[df['total_income'].isna(),'median_total_income']

# подсчёт пропусков после обработки
print('Пропуски после:', df['total_income'].isna().sum())

# удалим расчетный столбец
df = df.drop('median_total_income', 1)

Пропуски после: 0


**Вывод**

В результате исследования данных на пропуски было выявлено 2174 пропуска, пропуски присутствуют в двух колонках `days_employed` и `total_income` и присутствуют в строке одновременно, вероятно технический сбой и нужно указать на ошибку поставщику данных.
Пропуски были заполнены медианными значениями по группам **Семейный статус / Образование**.

### Замена типа данных

Согласно документации `days_employed` содержит данные в днях, поэтому необходимо их привести к целочисленному положительному формату.

In [12]:
# преобразуем days_employed в int
df['days_employed'] = df['days_employed'].astype('int')

# значения столбца перезапишем по модулю
df['days_employed'] = abs(df['days_employed'])

df[['dob_years', 'days_employed']].sort_values(by='days_employed', ascending=False)

,dob_years,days_employed
16467,56,401755
20108,69,401715
6158,61,401675
3899,60,401674
12956,61,401663
...,...,...
290,31,34
18583,43,33
12745,47,30
20045,32,24


В результате сортировки данных обнаружились аномально высокие данные, где стаж явно превосходит возраст,
сделаем предположение что в таких данных стаж записан в часах, пересчитаем стаж используя функцию.

In [13]:
def calculate_days_employed(row):
    """
    Если возраст меньше трудового стажа, то считаем что трудовой стаж записан в часах и переводим его в дни.
    Иначе значение не пересчитывается.
    """
    years = row['dob_years']
    days_employed = row['days_employed']
    if years < days_employed // 365:
        return days_employed // 24
    return days_employed

In [14]:
df['days_employed'] = df.apply(calculate_days_employed, axis=1)
df.sort_values(by='days_employed', ascending=False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
9769,1,18388,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4798,0,17615,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
16467,0,16739,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
20108,0,16738,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
6158,1,16736,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем


In [15]:
# преобразуем total_income в int
df['total_income'] = df['total_income'].astype('int')

**Вывод**

Столбец `days_employed` был преобразован к положительным int согласно документации к данным. Аномальные значения были пересчитаны и перезаписаны. Столбец `total_income` преобразован в int.

### Обработка дубликатов

In [16]:
# приведем данные всех столбцов с типом object к нижнему регистру
# только если данное преобразование сократит количество уникальных значений
for col in df.columns:
    if df.dtypes[col] == 'object':
        unique_before = df[col].value_counts().count()
        unique_after = df[col].str.lower().value_counts().count()
        if unique_before != unique_after:
            df[col] = df[col].str.lower()
            print('Привели к нижнему регистру данные столбца =',col)
            print('Количество уникальных значений до преобразования =', unique_before)
            print('Количество уникальных значений после преобразования =', unique_after)

Привели к нижнему регистру данные столбца = education
Количество уникальных значений до преобразования = 15
Количество уникальных значений после преобразования = 5


In [17]:
df[df.duplicated() == True].sort_values(by='total_income').head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16557,0,14069,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,127269,операции со своей недвижимостью
19018,0,1025,50,среднее,1,Не женат / не замужем,4,F,сотрудник,0,135824,недвижимость
19400,0,1025,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,135824,сделка с подержанным автомобилем
14690,0,1174,57,среднее,1,гражданский брак,1,M,пенсионер,0,136548,свадьба
14567,0,1174,45,среднее,1,гражданский брак,1,F,компаньон,0,136548,свадьба
12369,0,1174,58,среднее,1,гражданский брак,1,F,пенсионер,0,136548,сыграть свадьбу
12574,0,1174,60,среднее,1,гражданский брак,1,F,пенсионер,0,136548,свадьба
12672,0,1174,58,среднее,1,гражданский брак,1,F,пенсионер,0,136548,сыграть свадьбу
12958,0,1174,57,среднее,1,гражданский брак,1,F,пенсионер,0,136548,на проведение свадьбы
12975,0,1174,71,среднее,1,гражданский брак,1,F,пенсионер,0,136548,на проведение свадьбы


In [18]:
# подсчитаем явные дубликаты
print('До удаления:', df.duplicated().sum())
# удалим их
df = df.drop_duplicates().reset_index(drop=True)
# проверим результат
print('После удаления:', df.duplicated().sum())

До удаления: 71
После удаления: 0


**Вывод**

Столбец `education` содержал данные в разных регистрах, данные были приведены к нижнему регистру, для более корректного отображения справочника категории.

В результате поиска дубликатов было найдено и удалено 71 строк с данными.

Данные поставляются из разных источников, это можно понять из разных вариантов предоставления данных, например стаж в часах, а не в днях, или разные регистры в справочнике Образования, и вероятно это причина появления дубликатов.

### Лемматизация

In [19]:
m = Mystem()

# делаем уникальный список для Цели получения кредита
purpose_list = df['purpose'].unique()
print(purpose_list)

['покупка жилья' 'операции с жильем' 'покупка недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'покупка жилья для семьи' 'операции с коммерческой недвижимостью'
 'покупка коммерческой недвижимости' 'на покупку подержанного автомобиля'
 'сделка с автомобилем' 'жилье' 'автомобиль' 'профильное образование'
 'покупка жилья для сдачи' 'операции со своей недвижимостью' 'автомобили'
 'приобретение автомобиля' 'на покупку своего автомобиля'
 'покупка своего жилья' 'покупка жилой недвижимости'
 'заняться высшим образованием' 'получение высшего образования'
 'сделка с подержанным автомобилем' 'ремонт жилью' 'свой автомобиль'
 'строительство жилой недвижимости' 'высшее образование'
 'строительство недвижимости' 'операции с недвижимостью'
 'получение образования' 'образование' 'заняться образованием'
 'на покупку автомобиля' 'дополнительное образование'
 'получение дополнительного образования' 'сыграть свадьбу'
 'на проведение свадьбы' 'свадьба']


In [20]:
# каждое значение в 'purpose_list' лемматизируем, получаем список лемм и добавляем значения в словарь 'lemmas'
lemmas = {}
for i in purpose_list:
    lemma = m.lemmatize(i)
    for j in lemma:
        if j not in lemmas:
            lemmas[j] = 0
        lemmas[j] += 1
print(lemmas)

{'покупка': 10, ' ': 59, 'жилье': 7, '\n': 38, 'операция': 4, 'с': 5, 'недвижимость': 10, 'строительство': 3, 'собственный': 1, 'для': 2, 'семья': 1, 'коммерческий': 2, 'на': 4, 'подержать': 1, 'автомобиль': 9, 'сделка': 2, 'профильный': 1, 'образование': 9, 'сдача': 1, 'со': 1, 'свой': 4, 'приобретение': 1, 'жилой': 2, 'заниматься': 2, 'высокий': 3, 'получение': 3, 'подержанный': 1, 'ремонт': 1, 'дополнительный': 2, 'сыграть': 1, 'свадьба': 3, 'проведение': 1}


**Вывод**

Из полученных данных были собраны уникальные Цели и лемматизированы.
Из результата лемматизации выделено четыре категории.


### Категоризация данных

In [21]:
# разбиваем на категории по леммам: автомобиль, недвижимость, образование, свадьба и прочее
def purpose_category(purpose):
    lemmas_row = m.lemmatize(purpose)
    for i in lemmas_row:
        if 'свад' in i:
            return  'свадьба'
        if 'образов' in i:
            return 'образование'
        if 'авто' in i: 
            return 'автомобиль'
        if 'ремонт' or 'недвиж' or 'жил' in i:
            return  'недвижимость'
        return 'прочее'


df['purpose_category'] = df['purpose'].apply(purpose_category)
df['purpose_category'].value_counts()

недвижимость    19244
автомобиль        972
свадьба           791
образование       447
Name: purpose_category, dtype: int64

In [22]:
# добавим категорию по наличию детей
def children_category(children):
    if children == 0:
        return 'нет детей'
    return 'есть дети'


df['children_category'] = df['children'].apply(children_category)
df['children_category'].value_counts()

нет детей    14091
есть дети     7363
Name: children_category, dtype: int64

In [23]:
# напишем функцию для категоризации по доходу
def get_total_income_category(total_income):
    if total_income <= 30000:
        return '0 - 30000'
    elif total_income <= 60000:
        return '30001 - 60000'
    elif total_income <= 150000:
        return '60001 - 150000'
    elif total_income <= 500000:
        return '150001 - 500000'
    else:
        return 'более 500000'


df['total_income_category'] = df['total_income'].apply(get_total_income_category)
df['total_income_category'].value_counts()

60001 - 150000     10873
150001 - 500000     9553
30001 - 60000        784
более 500000         222
0 - 30000             22
Name: total_income_category, dtype: int64

**Вывод**

В результате применения функции для присвоения категории по Цели кредита все данные были категаризованы по четырем группам авто, недвижимость, образование и свадьбы. В категорию Прочее данных не попало.
Также добавили категорию по признаку наличия детей и по доходу.

По подсчету созданных признаков видно что кредиты больше всего берут на недвижимость, а также бездетные, меньше всего берут кредитов граждане с низким доходом

## Шаг 3. Ответьте на вопросы

In [24]:
# напишем функцию для рисования сводной таблицы

def draw_pivot_table(pdata, pindex, pvalues='debt', pfunc=['sum', 'count']):
    pdata_pivot = pdata.pivot_table(index=pindex, values=pvalues, aggfunc=pfunc)
    pdata_pivot['debt_conversion, %'] = round(100 * pdata_pivot.iloc[:, 0] / pdata_pivot.iloc[:, 1], 2)
    return pdata_pivot.sort_values(by='debt_conversion, %', ascending=False)

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [25]:
draw_pivot_table(df, 'children_category')

,sum,count,"debt_conversion, %"
,debt,debt,
children_category,,,
есть дети,678,7363,9.21
нет детей,1063,14091,7.54


**Вывод**

Из полученных данных можно сделать вывод, что чаще становятся имеющие детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
draw_pivot_table(df, 'family_status')

,sum,count,"debt_conversion, %"
,debt,debt,
family_status,,,
Не женат / не замужем,274,2810,9.75
гражданский брак,388,4151,9.35
женат / замужем,931,12339,7.55
в разводе,85,1195,7.11
вдовец / вдова,63,959,6.57


**Вывод**

Из полученных данных можно сделать вывод, что люди не женатые и в браке становятся чаще должниками, чем люди разведенные и вдовцы/вы.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
draw_pivot_table(df, 'total_income_category')

,sum,count,"debt_conversion, %"
,debt,debt,
total_income_category,,,
0 - 30000,2,22,9.09
60001 - 150000,938,10873,8.63
150001 - 500000,740,9553,7.75
более 500000,14,222,6.31
30001 - 60000,47,784,5.99


**Вывод**

Люди с маленькими доходами, чаще закрывают кредиты досрочно, чем те у которых доход средний или высокий.

- Как разные цели кредита влияют на его возврат в срок?

In [28]:
draw_pivot_table(df, 'purpose_category')

,sum,count,"debt_conversion, %"
,debt,debt,
purpose_category,,,
автомобиль,86,972,8.85
недвижимость,1559,19244,8.10
свадьба,64,791,8.09
образование,32,447,7.16


**Вывод**

Чаще досрочно выплачивают кредиты на авто/недвижимость, меньше за образование.

## Шаг 4. Общий вывод

Отвечая на вопрос влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.
Можно с уверенностью сказать что влияет, люди без детей и с узаконенными отношениями более ответственные заемщики.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.